### https://keras.io/examples/nlp/pretrained_word_embeddings/

In [1]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from pyspark.streaming.kafka import KafkaUtils
import pyspark.sql.functions as f
import json
import re
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import sys
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import lit

import pickle
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros
from keras.models import model_from_json
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
print(tf.__version__)

2.6.2


In [3]:
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras

# data_path = keras.utils.get_file(
#     "news20.tar.gz",
#     "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
#     untar=True,
# )

17342464/17329808 [==============================] - 46s 3us/step


In [3]:
csi_pos_neg = spark.read.csv("hdfs:///user/spark/datafile/csiposneg.csv", header=True)

In [4]:
csi_pos_neg = csi_pos_neg.drop('_c0')

In [5]:
csi_pos_neg.columns

['text', 'label']

In [4]:
# import os
# import pathlib

# data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
# dirnames = os.listdir(data_dir)
# print("Number of directories:", len(dirnames))
# print("Directory names:", dirnames)

# fnames = os.listdir(data_dir / "comp.graphics")
# print("Number of files in comp.graphics:", len(fnames))
# print("Some example filenames:", fnames[:5])

Number of directories: 20
Directory names: ['rec.sport.hockey', 'rec.motorcycles', 'talk.politics.guns', 'misc.forsale', 'talk.politics.mideast', 'soc.religion.christian', 'rec.autos', 'rec.sport.baseball', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'sci.space', 'comp.graphics', 'alt.atheism', 'comp.windows.x', 'sci.crypt', 'talk.religion.misc', 'talk.politics.misc', 'sci.electronics', 'comp.os.ms-windows.misc', 'sci.med']
Number of files in comp.graphics: 1000
Some example filenames: ['39029', '38957', '38379', '38592', '38332']


In [6]:
csi_pos_neg = csi_pos_neg.withColumn("label",col("label").cast("integer"))

In [7]:
# 우리가 해야할 일은 dataframe을 순서통일해서 text와 label 리스트로 나누는것
samples = csi_pos_neg.select("text").rdd.flatMap(lambda x: x).collect()
labels = csi_pos_neg.select("label").rdd.flatMap(lambda x: x).collect()

In [5]:
# samples = [] # text
# labels = [] # 클래스 라벨 개수 -> 우리의 경우는 classification 이니깐 2개가 되어야함
# # 추후에 multi label로 전환할 경우 여러개로
# class_names = []
# class_index = 0
# for dirname in sorted(os.listdir(data_dir)):
#     class_names.append(dirname)
#     dirpath = data_dir / dirname
#     fnames = os.listdir(dirpath)
#     print("Processing %s, %d files found" % (dirname, len(fnames)))
#     for fname in fnames:
#         fpath = dirpath / fname
#         f = open(fpath, encoding="latin-1")
#         content = f.read()
#         lines = content.split("\n")
#         lines = lines[10:]
#         content = "\n".join(lines)
#         samples.append(content)
#         labels.append(class_index)
#     class_index += 1

# print("Classes:", class_names)
# print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

In [8]:
# Shuffle the data
seed = 1337 # 난수 생성시 사용되는 시드number
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [9]:
from tensorflow.keras.layers import TextVectorization

# only consider the top 20,000 words, and will truncate or pad sequences to be actually 100 tokens long

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=100)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [10]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [11]:
path_to_glove_file = os.path.join(
    os.path.expanduser("~"), "/root/spark/glove.6B.100d.txt"
)

embeddings_index = {}
with open(path_to_glove_file, encoding="utf-8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [12]:
import csv
import numpy as np
from collections import Counter
from nltk.corpus import brown
from mittens import GloVe, Mittens
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
import csv
def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ',quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed
glove_path = "/root/spark/glove.6B.100d.txt"
pre_glove = glove2dict(glove_path)

In [75]:
#pre_glove와 embeddings_index는 같음

In [14]:
import nltk
nltk.download('brown')

sw = list(stop_words.ENGLISH_STOP_WORDS)
brown_data = brown.words()[:200000]
brown_nonstop = [token.lower() for token in brown_data if (token.lower() not in sw)]
oov = [token for token in brown_nonstop if token not in pre_glove.keys()]

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [15]:
# brown_data를 sample dataset으로 사용함 (continuous fine tunning을 위한)
brown_data

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [16]:
oov #이거를 생성할때 일정 frequency 미만인 단어는 지우는 로직을 구현하자

["atlanta's",
 'term-end',
 'presentments',
 'mayor-nominate',
 "georgia's",
 "georgia's",
 "department's",
 "ordinary's",
 "court's",
 'unmeritorious',
 "atlanta's",
 "commissioner's",
 "mayor's",
 "wife's",
 "mayor's",
 "mayor's",
 "governor's",
 "byrd's",
 "caldwell's",
 '$100',
 '$30',
 "georgia's",
 '$3',
 '$4',
 '$50',
 "body's",
 '$10',
 '87-31',
 "georgia's",
 "saturday's",
 "didn't",
 "didn't",
 "wasn't",
 "daniel's",
 "taxpayers'",
 '$451,500',
 '$157,460',
 "year's",
 '$88,000',
 "berry's",
 "we're",
 "i'm",
 'ex-gambler',
 "department's",
 '$1,000',
 '$12',
 "formby's",
 "couldn't",
 "texas'",
 '$5,000,000',
 '$15,000,000',
 "cotten's",
 "bill's",
 "boy's",
 "year's",
 "master's",
 'co-signers',
 "school's",
 "it's",
 "russia's",
 "karns'",
 'subpenas',
 'altho',
 'subpenaed',
 '$37',
 '$37',
 '$4,800',
 '$5,000',
 '$10',
 'hospital-care',
 '$20',
 "taxpayers'",
 "wouldn't",
 "president's",
 "nation's",
 "can't",
 '$1,500',
 '$2,000',
 '$1,000',
 '$1,500',
 "children's",
 '

In [17]:
def get_rareoov(xdict, val):
    return [k for (k,v) in Counter(xdict).items() if v<=val]

In [ ]:
# oov_rare = get_rareoov(oov, 1) # ferquency 1미만인것 지움
# corp_vocab = list(set(oov) - set(oov_rare))

In [ ]:
# brown_tokens = [token for token in brown_nonstop if token not in oov_rare]
# brown_doc = [' '.join(brown_tokens)]

In [18]:
corp_vocab = list(set(oov))
brown_doc = [' '.join(brown_nonstop)]

In [19]:
cv = CountVectorizer(ngram_range=(1,1), vocabulary=corp_vocab)
X = cv.fit_transform(brown_doc)
Xc = (X.T * X)
Xc.setdiag(0)
coocc_ar = Xc.toarray()

/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [20]:
mittens_model = Mittens(n=100, max_iter=1000) # n is embedding dimension
new_embeddings = mittens_model.fit(
    coocc_ar,
    vocab=corp_vocab,
    initial_embedding_dict= pre_glove)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Iteration 1000: loss: 0.02782272733747959

In [21]:
newglove = dict(zip(corp_vocab, new_embeddings))
f = open("repo_glove.pkl","wb")
pickle.dump(newglove, f)
f.close()

In [22]:
newglove # 새롭게 생성된 embeddings_index

{'swift-striding': array([-1.72410049e-02, -2.54893955e-03, -1.51560679e-02,  2.10264698e-03,
        -5.75124584e-02, -3.52296717e-02, -1.18833911e-02,  5.45300823e-03,
        -7.63049275e-02,  3.20129320e-02,  2.55985130e-02,  1.88834611e-02,
         8.31196308e-02, -1.23675819e-02, -1.50015522e-02, -6.97243363e-02,
         2.43763253e-03, -7.42693245e-02, -1.96479131e-02, -2.97167189e-02,
         5.35101816e-02,  7.64875710e-02,  4.74674627e-02,  7.36588463e-02,
        -8.29250962e-02, -1.97041631e-02, -7.49825388e-02, -2.11736634e-02,
        -3.38355079e-03,  1.22030135e-02,  4.70778979e-02,  9.20402482e-02,
        -2.09004264e-02, -8.78054276e-02,  2.52099521e-03,  1.41203403e-02,
        -6.75050691e-02, -5.93316928e-03,  3.76881324e-02, -2.33641677e-02,
         2.63647456e-02,  1.67002436e-02,  8.82390663e-02,  5.37936985e-02,
        -4.20354605e-02, -5.60910664e-02,  8.84434767e-03, -1.63146630e-02,
        -7.60056973e-02,  3.68714146e-02,  6.76566064e-02, -4.06564213

In [23]:
len(newglove)

2164

In [24]:
len(pre_glove)

400000

In [25]:
pre_glove.update(newglove)

In [26]:
len(pre_glove)

402164

In [27]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = pre_glove.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 16783 words (3217 misses)


In [28]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [29]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(2, activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         2000200   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         64128     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 128)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 128)         82048 

In [36]:
import tensorflow as tf
tf.disable_eager_execution()
print(tf.__version__)

AttributeError: module 'tensorflow' has no attribute 'disable_eager_execution'

In [34]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

RuntimeError: Attempting to capture an EagerTensor without building a function.

In [38]:
print(keras.__version__)

2.6.0


In [45]:
# 노드 하나에서만 동작함
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_val, y_val))

Epoch 1/100
1247/1247 [==============================] - 60s 48ms/step - loss: 0.1938 - acc: 0.9225 - val_loss: 0.1541 - val_acc: 0.9373
Epoch 2/100
1247/1247 [==============================] - 59s 48ms/step - loss: 0.1366 - acc: 0.9479 - val_loss: 0.1516 - val_acc: 0.9411
Epoch 3/100
1247/1247 [==============================] - 59s 48ms/step - loss: 0.1161 - acc: 0.9563 - val_loss: 0.1439 - val_acc: 0.9478
Epoch 4/100
1197/1247 [===========================>..] - ETA: 2s - loss: 0.1006 - acc: 0.9634

KeyboardInterrupt: 

In [35]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about computer graphics and 3D modeling"]]
)

class_names[np.argmax(probabilities[0])]

'comp.graphics'

In [37]:
class_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']